In [1]:
! python -m pip install -r requirements.txt

In [1]:
# Chroma client
import chromadb
chroma_client = chromadb.PersistentClient()

In [2]:
# create a test collection
collection_name = 'man_data'
chroma_client.create_collection(collection_name)

Collection(name=man_data)

In [3]:
test_collection = chroma_client.get_collection(collection_name);

In [4]:
# Add knowledge from a text file
from extract_text import file_chunks


text_file_path = './man_curl.txt'
texts = file_chunks(text_file_path)
texts[:10]

['curl(1)                           curl Manual                          curl(1)\n\nNAME\n       curl - transfer a URL\n\nSYNOPSIS\n       curl [options / URLs]\n\nDESCRIPTION\n       curl is a tool for transferring data from or to a server using URLs. It\n       supports these protocols: DICT, FILE, FTP, FTPS, GOPHER, GOPHERS, HTTP,\n       HTTPS, IMAP, IMAPS, LDAP, LDAPS, MQTT, POP3, POP3S, RTMP, RTMPS, RTSP,\n       SCP, SFTP, SMB, SMBS, SMTP, SMTPS, TELNET, TFTP, WS and WSS.\n\n       curl is powered by libcurl for all transfer-related features. See\n       libcurl(3) for details.\n\nURL\n       The URL syntax is protocol-dependent. You find a detailed description\n       in RFC 3986.\n\n       If you provide a URL without a leading protocol:// scheme, curl guesses\n       what protocol you want. It then defaults to HTTP but assumes others\n       based on often-used host name prefixes. For example, for host names\n       starting with "ftp." curl assumes you want FTP.\n\n       Yo

In [7]:
# Add knowledgebase from a PDF
from extract_pdf import extract_page_texts
pdf_path = 'pdf_path'
texts = extract_page_texts(pdf_path)
texts[:10]

Extracting 247


['No part of this publication may be reproduced without written permission.This publication includes the latest production information available before printing. Honda\nMotor Co., Ltd. reserves the right to make changes at any time without notice and withoutincurring any obligation.This manual should be considered a permanent part of the motorcycle and should remain\nwith the motorcycle when it is resold.\n2003 Honda Motor Co., Ltd.03/04/26 13:33:25 31MBZ600_001',
 '2004\nHonda CB600F599OWNER’S MANUAL03/04/26 13:33:28 31MBZ600_002',
 'ʵIntroduction\nIntroductionCongratulations on choosing your Honda\nmotorcycle.\nWhen you own a Honda, you’re part of a\nworldwide family of satisfied customers\npeople who appreciate Honda’s\nreputation for building quality into everyproduct.\nBefore riding, take time to get acquainted\nwith your motorcycle and how it works.To protect your investment, we urge youto take responsibility for keeping yourmotorcycle well maintained. Scheduledservice is a must,

In [5]:
from chromadb.api.client import Collection

def add_knowledge(collection: Collection, items: list[str], ids: list[str]):
    collection.add(
        documents=items,
        ids=ids,
    )

In [6]:
ids = [f'page-{index + 1}' for index in range(0, len(texts))]
add_knowledge(test_collection, texts, ids)

In [7]:
from chromadb.api.client import QueryResult

def print_results(results: QueryResult):
    print(f'Found in ids {results["ids"]}\n')
    print(f'Documents: \n')
    for documents in results['documents']:
        for document in documents:
            print(f'{document}\n')

In [9]:
results = test_collection.query(
    query_texts=["How do I add HTTP headers to a POST request in curl?"],
    n_results=3
)
print_results(results)

Found in ids [['page-47', 'page-48', 'page-103']]

Documents: 

You may specify any number of extra headers. Note that if you
              should add a custom header that has the same name as one of the
              internal ones curl would use, your externally set header is used
              instead of the internal one. This allows you to make even
              trickier stuff than curl would normally do. You should not
              replace internally set headers without knowing perfectly well
              what you are doing. Remove an internal header by giving a
              replacement without content on the right side of the colon, as
              in: -H "Host:". If you send the custom header with no-value then
              its header must be terminated with a semicolon, such as -H
              "X-Custom-Header;" to send "X-Custom-Header:".

              curl makes sure that each header you add/replace is sent with
              the proper end-of-line marker, you should t